In [ ]:
from collections import defaultdict
from pathlib import Path
import pandas as pd
import spacy
import tqdm
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#1738 samples

pow = "/content/drive/MyDrive/TFM/Albert/ArXiv_Papers"

results = defaultdict(list)
for file in Path(pow).iterdir():
    with open(file, "r",encoding='ISO 8859-1') as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())

In [ ]:
df = pd.DataFrame(results)

In [ ]:
nlp = spacy.load("en_core_web_sm")
stops = spacy.lang.en.stop_words.STOP_WORDS


def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)



In [ ]:
df['text_clean']=df['text'].progress_apply(normalize,lowercase=True, remove_stopwords=True)

NameError: ignored

In [ ]:
#df['text_clean']=df['text_clean'].str.join(' ')

In [ ]:
category_map = {'acc-phys':'Physics',
                'adap-org':'Physics',
                'alg-geom':'Maths',
                'ao-sci':'Physics',
                'astro-ph':'Physics',
                'atom-ph':'Physics', 
                'bayes-an':'Maths', 
                'chao-dyn':'Physics', 
                'chem-ph':'Physics', 
                'cmp-lg':'Computer Science', 
                'comp-gas':'Computer Science',
                'cond-mat':'Physics', 
                'cond-mat':'Physics',
                'cs':'Computer Science', 
                'dg-ga':'Computer Science',
                'econ':'Economics', 
                'eess':'Electrical Engineering and Systems Science', 
                'funct-an':'Maths',
                'gr-qc':'Physics',
                'hep-ex':'Physics',
                'hep-lat':'Physics',
                'hep-ph':'Physics', 
                'hep-th':'Physics', 
                'math':'Maths', 
                'math-ph':'Maths',
                'mtrl-th':'Maths',
                'nlin':'Physics', 
                'nucl-ex':'Physics',
                'nucl-th':'Physics', 
                'patt-sol':'Computer Science',
                'physics':'Physics',
                'plasm-ph':'Physics',
                'q-alg':'Maths', 
                'q-bio':'Quantitative Biology', 
                'q-fin':'Quantitative Finance', 
                'quant-ph':'Physics',
                'solv-int':'Maths',
                'stat':'Statistics',
                'supr-con':'Physics'}

In [ ]:
model=keras.models.load_model("/content/drive/MyDrive/TFM/Albert/OUTPUT_NAME/Paper_Classifier")

with open(r"/content/drive/MyDrive/TFM/Albert/tokenizer.pickle", "rb") as input_file:
  token = pickle.load(input_file)




#df.to_csv("Cleaned_Output.csv",index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
results=token.inverse_transform(model.predict_classes(df['text_clean']))


In [ ]:
rev_subs = { v:k for v,k in category_map.items()}
results=[rev_subs.get(item,item)  for item in results]
print (results)

['Computer Science', 'Quantitative Biology', 'Physics', 'Computer Science', 'Physics', 'Physics', 'Electrical Engineering and Systems Science', 'Computer Science', 'Physics', 'Computer Science', 'Electrical Engineering and Systems Science', 'Maths', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Electrical Engineering and Systems Science', 'Computer Science', 'Maths', 'Computer Science', 'Physics', 'Physics', 'Computer Science', 'Maths', 'Computer Science', 'Computer Science', 'Computer Science', 'Physics', 'Computer Science', 'Maths', 'Maths', 'Computer Science', 'Computer Science', 'Physics', 'Maths', 'Physics', 'Maths', 'Computer Science', 'Computer Science', 'Electrical Engineering and Systems Science', 'Computer Science', 'Physics', 'Electrical Engineering and Systems Science', 'Maths', 'Maths', 'Computer Science', 'Physics', 'Physics', 'Computer Science', 'Computer Sc

In [ ]:
#df['category']=results
df[df['file_name']=='2106.0349_Quantum_Error_Correction_and_Holographic_Information_from_Bilocal_Holography.pdf.txt']
#df['file_name'].unique()

,file_name,text,text_clean
1767,2106.0349_Quantum_Error_Correction_and_Hologra...,arXiv:2106.00349v1 [hep-th] 1 Jun 2021\n\nQuan...,arxiv:2106.00349v1 [ hep - th ] 1 jun 2021 qua...


In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model_bert = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
import scipy
query = "We know that the bilocal collective field theory will reproduce the CFT correlators"#@param {type: "string"}

query=normalize(query,lowercase=True, remove_stopwords=True)
queries = [query]
query_embeddings = model_bert.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches =  1#@param {type: "number"}

dropdown = 'Physics' #@param ['Physics','Maths','Economics','Computer Science','Quantitative Biology','Quantitative Finance','Statistics','Electrical Engineering and Systems Science','Economics']

sentences = df['text_clean'][df['category']==dropdown].values.tolist()

sentence_embeddings = model_bert.encode(sentences)
print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    #print(type(results))

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {0} files where the input matches the corpus for {1} category are:".format(number_top_matches,dropdown))
    print("\n\n======================\n\n")
    for idx, distance in results[0:number_top_matches]:
      #if (1-distance)>0.5:
        if (distance)>0.8:  
          print(df['file_name'].iloc[[idx]].to_list(),"(Cosine Score: %.4f)" % (1-distance))
          #print(sentences[idx].strip())
          #print()
          print("\n\n======================\n\n")